# Access Token 생성
- 학습된 모델을 Huggingface hub에 올리기 위해서는 access token이 필요하다.
  
![huggingface_create_apikey.png](figures/huggingface_accesstoken.png)
![huggingface_create_apikey.png](figures/huggingface_accesstoken2.png)

- 1. 로그인 -> 2. Profile -> 3. Access Tokens 선택
- 생성할 때 `write` 권한을 선택한다.

In [ ]:
# working directory 아래 ".env" 파일을 생성
# .gitignore 파일에 .env 을 등록해서 github에 등록하지 않도록 설정.
# 이름 = 값 형식으로 환경변수를 설정.
# HUGGINGFACE_API_KEY = "받은 Access Token"
# 추후에 api사용할시에 같은 방식으로 작업.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
# working directory에서 환경변수 파일 (defalut = ".env")를 찾아서 
# 그 파일에 설정된 값을 O/S의 환경변수로 등록해준다.
# 임시적으로 사용하는 것이기 때문에 프로그램 종료시 삭제.

True

In [3]:
# 환경 변수 값을 읽기
import os
os.getenv("JAVA_HOME")

'C:\\Users\\tjddm\\Documents\\jdk-24.0.1'

In [ ]:
hf_api_key = os.getenv("HUGGINGFACE_API_KEY")
# os.environ["HUGGINGFACE_API_KEY"]

# Naver 영화댓글 분류

# Huggingface Dataset 패키지
- 허깅페이스 허브에 공유된 데이터셋을  다운로드해서 전처리 및 관리할 수있도록 돕는 라이브러리. 
- 많은 공개데이터셋을 동일한 인터페이스로 사용할 수있다.
- 설치
    - `pip install datasets`
- https://huggingface.co/datasets
- https://github.com/huggingface/datasets
      
## Huggingface Dataset loading
- datasets 로딩
    - `load_data('dataset name')`
        - huggingface datasets에 등록된 Dataset 이름 넣어 Loading한다.
          
![img](figures/huggingface_dataset.png)

In [7]:
# NSMC 데이터셋 로딩
import datasets
from datasets import load_dataset
# dataset download시에 데이터를 올려놓거나 / 다운로드받는 코드를 올려둠.(악성코드등이 같이 실행될수있음.)

nsmc = load_dataset("e9t/nsmc", trust_remote_code=True)

README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

c:\Users\tjddm\miniconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tjddm\.cache\huggingface\hub\datasets--e9t--nsmc. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


nsmc.py:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
# Dataset : 데이터 셋
# Datasetdict : Dataset들을 모아놓은 dict기반의 자료구조. 
# -> train/valid/test set들을 모아서 제공할 때 사용

In [9]:
nsmc.keys()

dict_keys(['train', 'test'])

In [ ]:
# Dataset이 표데이터일 경우 features는 칼럼명.
trainset = nsmc["train"]
testset = nsmc["test"]

In [ ]:
# feature 조회
trainset['document'][:5] # 댓글내용
trainset['label'][:5] # 긍정 / 부정
trainset['id'][:5] # 댓글 id

['9976970', '3819312', '10265843', '9045019', '6483659']

In [17]:
# dataset.Datasets를 다른 형식으로 변환
# dataset객체.to_xxxx()

df = trainset.to_pandas()
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [18]:
# 다른 형식으로 저장된 data들을 dataset.Datasets로 변환
# dataset객체.from_xxxx()

d = datasets.Dataset.from_pandas(df.head(100))
d

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 100
})

In [91]:
# Sampling - train : 10000, test : 5000

# dataset.shuffle() -> 섞어줌. select([조회할 index들])
sample_train = trainset.shuffle().select(range(10000))
sample_test = testset.shuffle().select(range(5000))

In [92]:
print(sample_train)
print(sample_test)

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 10000
})
Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 5000
})


## 모델, 토크나이저 loading

- 모델 별 Model 클래스를 이용하거나 Auto class를 이용해 모델, 전처리기(tokenizer, ImageProcessor 등)을 로딩한다.
    - Huggingface에 저장된 model name을 입력해서 pretrained 모델을 loading 한다.
    - fine tuning 한 경우 모델 저장 디렉토리 경로를 넣어 pretrained 모델을 loading한다.
- AutoModel은 model name을 주면 그 모델이 학습한 base 모델에 맞는 객체를 생성해서 반환한다.
    - Auto Model은 task 별로 다양한 클래스들이 있다.
        - 클래스 이름 형식: AutoModelFor{Task형식}
        - ex) `AutoModelForObjectDetection`, `AutoModelForSequenceClassification`
    - https://huggingface.co/docs/transformers/model_doc/auto
    - 전처리기(tokenzier)는 사용하려는 모델이 사용한 전처리기를 사용해야 한다.

In [93]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "beomi/kcbert-base"
# 토크나이저, 분류 모델 loading
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels = 2) 
# num_labels : 분류할 클래스의 개수 (긍정, 부정) -> 학습안된 분류기 추가해서 학습시키기.
# model_id : kcbert-base - pretrainde 된 Feature Extractor
# model : Estimator는 학습 안된 layer로 구성

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# model_id = "beomi/kcbert-base" -> 문장의 특성을 추출하는 모델. (분류 모델 X)
# pretrained된 모델 파인튜닝 + 모델에서 추출된 feature을 토대로 초기화된 분류기를 학습시켜서 결과 추출하기.

In [23]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## pytorch Dataset 생성
모델 입력으로 다음 4개 항목을 dictionary로 묶어서 제공하도록 구현한다.
1. input_ids: 입력 text 토큰을 id로 변환한 값
2. token_type_ids: 문자쌍 구분시 사용. 단일 문장: 0, 문자쌍-첫문장: 0, 두 번째 문장: 1
3. attention_mask: 실제 토큰값과 패딩구분값
4. labels: 정답 class index

1 ~ 3은 위의 train_encoding, test_encoding으로 만듬. labels은 train_data/test_data의 label 키 값 사용

In [94]:
train_X = sample_train['document'] # sample_trian -> trainset(전체)
train_y = sample_train['label']

test_X = sample_test['document'] # sample_test -> testset(전체)
test_y = sample_test['label']
# trainset/testset 사용하지 않는 이유는 전체 데이터를 사용하지 않기 위함

In [95]:
# X(댓글)을 토큰화
train_encoding = tokenizer(train_X, return_tensors="pt", padding=True) # trian_X에서 가장 긴 문장을 기준으로 [PAD] 추가.
test_encoding = tokenizer(test_X, return_tensors="pt", padding=True)

In [96]:
train_encoding.keys()
train_encoding['input_ids'].shape # [10000:문서수, 142:토큰수]

torch.Size([10000, 110])

In [113]:
##################################
# Pytorch의 Dataset을 정의, 생성
##################################

from torch.utils.data import Dataset
import torch

class NSMCDataset(Dataset):

    def __init__(self, comments, labels):
        """
        Args:
            comments(dict) : tokenizer로 토큰화한 input data들
            labels(list) : 정답 라벨들
        """
        self.comments = comments
        self.labels = labels

    def __len__(self):
        return len(self.comments['input_ids'])

    def __getitem__(self,index):
        """
        index번째 데이터를 반환.
        BERT 모델 입력 형식에 맞춰서 반환. input_ids, token_type_ids, attention_mask + label

        Args:
            index(int) :
        Returns:
            directory - input_ids, token_type_ids, attention_mask + label
            입력 데이터와 정답 label을 딕셔너리에 묶어서 반환.
        """
        data = {key:value[index] for key, value in self.comments.items()}
        # comments.items -> (key, [value1, value2, ... value10000])
        data['labels'] = torch.tensor([self.labels[index]], dtype=torch.int64)

        return data


In [114]:
train_set = NSMCDataset(train_encoding, train_y)
test_set = NSMCDataset(test_encoding, test_y)

In [115]:
len(train_set), len(test_set)

(10000, 5000)

# 학습
- Transformers는 model 학습을 위해 TrainingArguments, Trainer 클래스를 제공한다.
- TrainingArguments는 Trainer를 위한 설정을 하는 클래스
- TrainingArguments, Trainer를 이용하면 training option, logging, gradient accumulation, mixed precision등을 쉽게 설정해 학습, 평가를 모두 진행할 수 있다.

In [ ]:
from transformers import TrainingArguments, Trainer

N_EPOCHS = 1
BATCH_SIZE = 64

# 어떻게 학습할지에 대한 설정.
train_args = TrainingArguments(
    output_dir="models/nsmc", # train 모델을 저장할 디렉토리 경로.
    num_train_epochs=N_EPOCHS, # 학습 에폭수.
    per_device_train_batch_size=BATCH_SIZE, # 학습할때 batch size.
    per_device_eval_batch_size=BATCH_SIZE, # 검증시 batch size.
    
    eval_strategy="epoch", # 평가 전략. - "no", "step", "epoch" -> step 단위별로 평가.
    save_strategy="epoch", # 저장 전략. -> epoch단위별로 저장. / "no" -> 아무것도 안함.

    save_total_limit=1, # 저장할 모델의 최대 개수. -> 마지막것만 저장.
    load_best_model_at_end=True, # 학습 종료 후 검증결과가 가장 좋은 모델을 저장 + load.
                                 # True : eval_strategy, save_strategy가 같아야함.

    metric_for_best_model="eval_loss", # best model을 선정할 검증 기준이 될 평가 지표.
    greater_is_better=False, # 검증 평가지표값이 높아야 좋은지 낮아야 좋은지. / False -> 낮을때 사용

    report_to="none"

    # huggingface-hub에 올리는 설정
    # push_to_hub=True,
    # hub_model_id="kcbert-nsmc-10000" # 모델_id
    # hub_token = "Access Token"
    # -> 학습이 끝나면 자동으로 huggingface 업로드

)

In [117]:
# 평가 함수 정의 - evaluate 패키지를 이용.
# huggingface에서 제공하는 라이브러리. / 다양한 평가 함수들을 제공.

In [118]:
import evaluate

# 정확도 평가 함수.
acc_fn = evaluate.load("accuracy") # f1, recall, precision ...

In [ ]:
# 사용 예시

pred = torch.tensor([0,1,0,1])
ref = torch.tensor([0,1,0,0])

acc_fn.compute(predictions=pred, references=ref)

{'accuracy': 0.75}

In [122]:
def compute_metrics(pred):
    """
    모델이 학습하는 도중 예측값을 받아서 평가 점수를 계산하는 함수.
    Trainer에 의해서 호출될 함수.
    Args:
        pred(EvalPrediction) - 모델의 예측값, 정답을 묶어서 제공.
    Return:
        dictionary - key : 평가지표이름, value : 평가점수
    """
    labels = pred.label_ids # 정답
    preds = pred.predictions.argmax(axis=-1) # 모델의 예측값.
    metrics1 = evaluate.load("accuracy")
    metrics2 = evaluate.load("f1")

    acc = metrics1.compute(predictions=preds, references=labels)
    f1 = metrics2.compute(predictions=preds, references=labels)

    return {"accuracy" :acc, "f1 score":f1}



In [123]:
# Trainer 객체

trainer = Trainer(
    model=model, # 학습 대상 모델
    args=train_args, #TrainingArguments
    train_dataset=train_set, # 학습 데이터셋, pytorch의 Dataset객체 - trianer.train()
    eval_dataset=test_set, # 검증 데이터셋 trainer.evaluate()
    compute_metrics=compute_metrics # loss 이외에 검증/평가시 확인할 지표를 계산하는 함수.
)

In [121]:
# 학습

trainer.train()

c:\Users\tjddm\miniconda3\envs\dl\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [125]:
# 저장 - local directory
# tokenizer와 model을 같은 경로에 저장.

save_path = "models/nsmc"
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

## 모델 로드

In [127]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

load_tokenizer = AutoTokenizer.from_pretrained(save_path)
load_model = AutoModelForSequenceClassification.from_pretrained(save_path)

# 추론

In [126]:
sentence = ["이걸 영화라고 만든 거냐?", "아무 기대 없이 봤는데 재미있네.", "내가 감독이어도 이것보다 재미있게 만들겠다.", "시간이 어떻게 가는 줄 모르고 봤다."]

In [128]:
from transformers import pipeline

pipe = pipeline(task="text-classification", tokenizer=load_tokenizer, model=load_model)
result = pipe(sentence)

Device set to use cpu


In [129]:
result

[{'label': 'LABEL_0', 'score': 0.9500664472579956},
 {'label': 'LABEL_1', 'score': 0.869631290435791},
 {'label': 'LABEL_0', 'score': 0.526900053024292},
 {'label': 'LABEL_1', 'score': 0.5647743344306946}]

In [130]:
# 학습한 모델, 토크나이저를 huggingface hub에 업로드.
# 1. TrainingArguments에 설정해서 학습 도중 / 끝나면 자동으로 업로드 되게 설정.
# 2. model/tokenizer.push_to_hub("계정/모델id")를 통해서 수동으로 올린다.

# 로그인
from huggingface_hub import login
from dotenv import load_dotenv
# login() # Token에 Access Token 입력 후 Login 클릭
# login("Huggingface Access Token") # 바로 입력
load_dotenv() # .env에 환경 변수들을 로드.

True

In [ ]:
import os
hf_api_key = os.getenv("HUGGINGFACE_API_KEY")
login(hf_api_key)

In [133]:
model_id = "kcbert-nsmc-10000"
load_tokenizer.push_to_hub(model_id)
load_model.push_to_hub(model_id)



README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\tjddm\miniconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tjddm\.cache\huggingface\hub\models--seongui--kcbert-nsmc-10000. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/seongui/kcbert-nsmc-10000/commit/1ca40c97bab03e090a6ae93a56f3b87179d477ae', commit_message='Upload BertForSequenceClassification', commit_description='', oid='1ca40c97bab03e090a6ae93a56f3b87179d477ae', pr_url=None, repo_url=RepoUrl('https://huggingface.co/seongui/kcbert-nsmc-10000', endpoint='https://huggingface.co', repo_type='model', repo_id='seongui/kcbert-nsmc-10000'), pr_revision=None, pr_num=None)

In [135]:
# hugging face에 push한 모델 사용.
model_id = "kgmyh/kcbert-nsmc-10000"
pipe = pipeline(task="text-classification", tokenizer=model_id, model=model_id)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

c:\Users\tjddm\miniconda3\envs\dl\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tjddm\.cache\huggingface\hub\models--kgmyh--kcbert-nsmc-10000. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/721k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cpu


In [137]:
pipe(["영화 너무 재미있어요", "시간이 아깝다."])

[{'label': 'LABEL_1', 'score': 0.9964871406555176},
 {'label': 'LABEL_0', 'score': 0.9996107220649719}]